create column ValveMask_label depending on value of ValveMask, Delta_18_16, and  Delta_D_H

DI  : δ18O(Delta_18_16):  -7.78 ± 0.01 permil; δD(Delta_D_H):  -50.38 ± 0.02 permil  
GSM1: δ18O(Delta_18_16): -33.07 ± 0.02 permil; δD(Delta_D_H): -262.95 ± 0.04 permil

In [1]:
#%matplotlib inline

# import matplotlib
# import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import ValveMaskconverter as VMconvert

In [2]:
def add_ValveMask_label(filename_, fparam_):
    """
    """
    fpath = Path(filename_)

    std = VMconvert.Standard(fparam_)
    instrument= str(fpath.name).split('-')[0]
    std.get(instrument)

    # Read data from file 'filename.csv'
    # (in the same directory that your python process is based)
    # Control delimiters, rows, column names with read_csv (see later)
    df = pd.read_csv(fpath, sep='\s+', parse_dates=[['DATE', 'TIME']])
    # add 'rank' column
    # consecutive lines with the same 'ValveMask' value, get the same rank
    df['rank'] = ((df.ValveMask != df.ValveMask.shift()).cumsum())
    # compute median for each group of same rank
    df['Delta_18_16_median'] = df['Delta_18_16'].groupby(df['rank']).transform('median')
    df['Delta_D_H_median'] = df['Delta_D_H'].groupby(df['rank']).transform('median')
    # add column 'label', value depending on standard range
    # df['ValveMask_label'] = df.apply (lambda row: label_ValveMask(row, std), axis=1)
    df['ValveMask_label'] = df.apply (lambda row: std.label_ValveMask(row), axis=1)
    # remove temporary column, previously added
    # df.drop(['rank', 'Delta_18_16_median', 'Delta_D_H_median'], axis=1, inplace=True)

    return df, std

In [11]:
from bokeh.io import reset_output, output_notebook, output_file, show, save
from bokeh.layouts import column, row, gridplot
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, BoxAnnotation, Band
from bokeh.models import HoverTool, ResetTool, TapTool
from bokeh.models import PanTool,BoxZoomTool,WheelZoomTool
# from bokeh.transform import linear_cmap
# from bokeh.palettes import Spectral7
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

fparam='parameters.yaml'

def plot(filename_, save_=False, notebook_=False, extra=None):
    
    fpath = Path(filename_)
    df, std = add_ValveMask_label(fpath, fparam)
    
    if extra is not None:
        dff = df[['DATE_TIME','Delta_18_16','Delta_D_H',
                  'Delta_18_16_median','Delta_D_H_median',
                  'ValveMask','ValveMask_label',extra]]
    else:
        dff = df[['DATE_TIME','Delta_18_16','Delta_D_H',
                  'Delta_18_16_median','Delta_D_H_median',
                  'ValveMask','ValveMask_label']]
    
    refdf = dff[['DATE_TIME',extra]]
    refdf = refdf.assign( Delta_18_16_DI=std.iso['d18O']['DI']['unpacked'],
                 Delta_18_16_DI_packed=std.iso['d18O']['DI']['packed'],
                 Delta_18_16_DI_lower=std.range('d18O','DI')[0],
                 Delta_18_16_DI_upper=std.range('d18O','DI')[1],
                 Delta_18_16_GSM1=std.iso['d18O']['GSM1']['unpacked'],
                 Delta_18_16_GSM1_packed=std.iso['d18O']['GSM1']['packed'],
                 Delta_18_16_GSM1_lower=std.range('d18O','GSM1')[0],
                 Delta_18_16_GSM1_upper=std.range('d18O','GSM1')[1],
                 Delta_D_H_DI=std.iso['dD']['DI']['unpacked'], 
                 Delta_D_H_DI_packed=std.iso['dD']['DI']['packed'], 
                 Delta_D_H_DI_lower=std.range('dD','DI')[0],
                 Delta_D_H_DI_upper=std.range('dD','DI')[1],
                 Delta_D_H_GSM1=std.iso['dD']['GSM1']['unpacked'],
                 Delta_D_H_GSM1_packed=std.iso['dD']['GSM1']['packed'],
                 Delta_D_H_GSM1_lower=std.range('dD','GSM1')[0],
                 Delta_D_H_GSM1_upper=std.range('dD','GSM1')[1])  
    
    
    reset_output()
    if notebook_:
        output_notebook()
    htmlout = fpath.parents[0] / Path(fpath.stem+'.html')
    output_file(htmlout)

    datasource = ColumnDataSource(dff)
    refsource = ColumnDataSource(refdf)

    # ==== create table ====
    # TableColumn(field="dates", title="Date", formatter=DateFormatter())

    #Columns = [TableColumn(field=Ci, title=Ci) for Ci in dff.columns] # bokeh columns
    Columns = [TableColumn(field=Ci, title=Ci, formatter=DateFormatter(format="%m/%d/%Y %H:%M:%S")) 
               if Ci == 'DATE_TIME' else TableColumn(field=Ci, title=Ci) for Ci in dff.columns]
    t = DataTable(columns=Columns, 
                  source=datasource, 
                  width=900, height=280,
                  fit_columns=True,
                  editable=False,
                  scroll_to_selection= True
                  #selectable = True
                  ) # bokeh table

    # === create plots =======
    left = dff.index[dff.ValveMask != dff.ValveMask.shift()].to_list()
    right = [x-1 for x in left[1:]]
    right.append(len(dff)-1)

    # == plot Delta_D_H ==
    dlist=['Delta_D_H','Delta_18_16']
    refcolor=[('DI','cyan'),('GSM1','lime')]
    p=[]
    hover={}
    # tools=[PanTool(),BoxZoomTool(),WheelZoomTool(),TapTool(), ResetTool()]
    for idx, dname in enumerate(dlist):

        # define figure
        p.append(figure(x_axis_type="datetime", x_axis_label='Time', y_axis_label='Value', 
                        toolbar_location="above",
                        plot_width=900, plot_height=400,
                        title=dname
                        #tools=tools
                       ))
        if idx != 0:
            p[idx].x_range=p[0].x_range

        for refname, color in refcolor:
    
            # add band around ref(s) value
            vname = '_'.join([dname,refname])
            ref= p[idx].line('DATE_TIME', vname, color=color, name=vname, line_width=2, alpha=0.5, source=refsource)
            p[idx].varea(x='DATE_TIME', y1=vname+'_lower', y2=vname+'_upper', source=refsource,
                    fill_alpha=0.1, fill_color=color, legend_label=refname)
            p[idx].line('DATE_TIME', vname+'_packed', color=color, name=vname+'_packed', line_width=2, line_dash='dashed', alpha=0.5, source=refsource)

        # create lines
        d1 = p[idx].line('DATE_TIME', dname, name=dname, color="olive", line_width=2, alpha=0.5, source=datasource)
        d2 = p[idx].line('DATE_TIME', dname+'_median', color="coral", line_dash='dashed', line_width=2, alpha=0.5, source=datasource)
        for l,r in list(zip(left,right)):
            if dff['ValveMask'][l] == 6:
                box = BoxAnnotation(left = dff['DATE_TIME'][l], right = dff['DATE_TIME'][r], fill_alpha = 0.2)
                p[idx].add_layout(box)
                
        if extra is not None:
            # add extra variable
            p[idx].line('DATE_TIME', extra, color='red', name=extra, line_width=2, source=refsource)
        

        # Assign the legend to the bottom left: p.legend.location
        p[idx].legend.location = 'bottom_left'
        # Fill the legend background with the color 'lightgray': p.legend.background_fill_color
        # p[idx].legend.background_fill_color = 'gray'

        # add hover tools to the figure
        hover[0] = HoverTool(
            renderers=[d1],
            # you can see the detail of the formatter at:
            # https://bokeh.pydata.org/en/latest/docs/reference/models/formatters.html
            tooltips=[('Time','@DATE_TIME{%F %H:%M}'),
                  ('Name','$name'),
                  ('Value', '@$name'),
                  ('median', '@Delta_D_H_median'),
                  ('label', '@ValveMask_label')
                 ],
            formatters={'@DATE_TIME':'datetime',},
            # display a tooltip whenever the cursor is vertically in line with a glyph
            mode='vline'
            )
        #
        hover[1] = HoverTool(
            renderers=[d1],
            # you can see the detail of the formatter at:
            # https://bokeh.pydata.org/en/latest/docs/reference/models/formatters.html
            tooltips=[('Time','@DATE_TIME{%F %H:%M}'),
                  ('Name','$name'),
                  ('Value', '@$name'),
                  ('median', '@Delta_18_16_median'),
                  ('label', '@ValveMask_label')
                 ],
            formatters={'@DATE_TIME':'datetime',},
            # display a tooltip whenever the cursor is vertically in line with a glyph
            mode='vline'
            )  

        p[idx].add_tools(hover[idx])

    # put the results in a column and show
    # show(column(p1))
    # make a grid
    # grid = gridplot([[s1, s2], [None, s3]], plot_width=250, plot_height=250)
    layout = gridplot([[t], [p[0]], [p[1]]])
    
    if save_:
        save(layout)
    else:
        show(layout)

In [ ]:
!ls /home/jpa029/Data/Quince/sample_files/*.dat

In [ ]:
filelist=['/home/jpa029/Data/Quince/sample_files/HIDS2254-20190104-220043Z-DataLog_User.dat',
          '/home/jpa029/Data/Quince/sample_files/HIDS2254-20190718-095023Z-DataLog_User.dat',
          '/home/jpa029/Data/Quince/sample_files/HKDS2039-20190207-085422Z-DataLog_User.dat'
         ]

In [ ]:
for dat in filelist:
    plot(dat, save_=True)

In [9]:
dat="/home/jpa029/Data/Quince/sample_files/HKDS2039-20190207-085422Z-DataLog_User.dat"

In [26]:
# 'OutletValve'
plot(dat,notebook_=True, extra='residuals')

Loading BokehJS ...

In [6]:
fpath = Path(dat)
df = pd.read_csv(fpath, sep='\s+', parse_dates=[['DATE', 'TIME']])

In [7]:
df.head()

,DATE_TIME,FRAC_DAYS_SINCE_JAN1,FRAC_HRS_SINCE_JAN1,JULIAN_DAYS,EPOCH_TIME,ALARM_STATUS,INST_STATUS,CavityPressure,CavityTemp,WarmBoxTemp,...,strength1,str1_offset,strength2,str2_offset,strength3,str3_offset,strength_11,str11_offset,strength_13,str13_offset
0,2019-02-07 08:54:16.802,37.371028,896.904667,38.371028,1.549530e+09,0,963,49.997438,79.998589,44.999931,...,2080.903267,0.0,1238.431410,0.0,167.020992,0.0,0.0,0.0,0.0,0.0
1,2019-02-07 08:54:17.631,37.371037,896.904898,38.371037,1.549530e+09,0,963,49.996810,79.998589,44.999931,...,2081.310643,0.0,1238.409390,0.0,166.904739,0.0,0.0,0.0,0.0,0.0
2,2019-02-07 08:54:18.457,37.371047,896.905127,38.371047,1.549530e+09,0,963,50.012170,79.998589,44.999931,...,2081.552909,0.0,1238.582390,0.0,167.007509,0.0,0.0,0.0,0.0,0.0
3,2019-02-07 08:54:19.283,37.371057,896.905357,38.371057,1.549530e+09,0,963,50.016999,79.998567,44.999921,...,2080.918476,0.0,1238.261656,0.0,166.837538,0.0,0.0,0.0,0.0,0.0
4,2019-02-07 08:54:20.105,37.371066,896.905585,38.371066,1.549530e+09,0,963,50.005733,79.998528,44.999901,...,2080.667414,0.0,1238.017150,0.0,166.788117,0.0,0.0,0.0,0.0,0.0
